# Dogs vs Cats

原题目地址：https://www.kaggle.com/c/dogs-vs-cats

### 题目要求
训练一个分类器，识别图片是中是猫还是狗。

<img src="./images/pics.jpg" width="400">

## 下载数据并解压

### 关于数据集
网络服务，如邮件、blog等，为了防止滥发广告，一般都要求用验证码 CAPTCHA (Completely Automated Public Turing test to tell Computers and Humans Apart) or HIP (Human Interactive Proof) 来证明是人类操作而不是机器人。

[Asirra](http://research.microsoft.com/en-us/um/redmond/projects/asirra/) (Animal Species Image Recognition for Restricting Access)是一份让人类来判断图片是猫是狗的验证码数据集。

题目提供的数据集分为训练集和测试集([网盘下载](https://pan.baidu.com/))。其中，训练集包含 x 张图片，其中猫 x1 张，狗 x2 张。测试集包含 y 张图片。

图片下载后，解压到 ```Dogs_vs_Cats/images``` 目录。

### 训练

In [ ]:
import glob
import os

import tflearn
from sklearn.model_selection import train_test_split

pjoin = os.path.join
TRAIN_DATA = pjoin(os.path.dirname(__file__), 'images')
MODEL_PATH = pjoin(os.path.dirname(__file__), 'model')
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH, mode=0755)
MODEL_NAME = 'resnet_dogs_vs_cats.model'

import numpy as np
from skimage import transform
from skimage.io import imread

MEAN_PIXEL = [127, 127, 127]


# 标准化图片，将图片缩放到固定尺寸，做适当剪裁
def crop_image(image, dshape):
    factor = float(min(dshape[:2])) / min(image.shape[:2])
    new_size = [int(image.shape[0] * factor), int(image.shape[1] * factor)]
    if new_size[0] < dshape[0]:
        new_size[0] = dshape[0]
    if new_size[1] < dshape[0]:
        new_size[1] = dshape[0]
    resized_image = transform.resize(image, new_size)
    sample = np.asarray(resized_image) * 256
    if dshape[0] < sample.shape[0] or dshape[1] < sample.shape[1]:
        xx = int((sample.shape[0] - dshape[0]))
        yy = int((sample.shape[1] - dshape[1]))
        xstart = xx / 2
        ystart = yy / 2
        xend = xstart + dshape[0]
        yend = ystart + dshape[1]
        sample = sample[xstart:xend, ystart:yend, :]
    return sample


# 读入图片，预处理
#  1. 调整图片尺寸
#  2. 白化，将图片像素值从 [0, 255] 转换成 [-127, 128]，即像素均值为 0
def preprocess_image(path, dshape):
    image = imread(path)
    image = crop_image(image, dshape=dshape)
    image -= MEAN_PIXEL
    return image


# 批量读入数据
# 此处小数据直接读入内存
def read_data():
    X = []
    Y = []
    for f in glob.glob(TRAIN_DATA + '/*.jpg'):
        fname = os.path.basename(f)
        # 0 for cat, 1 for dog
        label = 0 if fname.startswith('cat') else 1
        image = preprocess_image(f, [256, 256, 3])
        X.append(image)
        Y.append(label)
    # split training data and validation set data
    X, X_test, y, y_test = train_test_split(X, Y,
                                            test_size=0.2,
                                            random_state=42)
    return (X, y), (X_test, y_test)


# 构建ResNet模型
def resnet():
    # Residual blocks
    n = 5

    # Building Residual Network
    net = tflearn.input_data(shape=[None, 256, 256, 3])
    net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.residual_block(net, n, 16)
    net = tflearn.residual_block(net, 1, 32, downsample=True)
    net = tflearn.residual_block(net, n - 1, 32)
    net = tflearn.residual_block(net, 1, 64, downsample=True)
    net = tflearn.residual_block(net, n - 1, 64)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'relu')
    net = tflearn.global_avg_pool(net)
    # Regression
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',
                             loss='categorical_crossentropy')
    return net


# 完整的训练过程
def train():
    # X, Y 为训练集
    # X_test, Y_test 为验证集
    (X, Y), (X_test, Y_test) = read_data()
    Y = tflearn.data_utils.to_categorical(Y, 2)
    Y_test = tflearn.data_utils.to_categorical(Y_test, 2)

    # 模型
    model = tflearn.DNN(resnet(), checkpoint_path='model_resnet',
                        max_checkpoints=10, tensorboard_verbose=0,
                        clip_gradients=0.)

    # 若之前有训练过，直接加载之前训练的中间结果，加载后可以继续训练
    if os.path.exists(pjoin(MODEL_PATH, MODEL_NAME)):
        model.load(pjoin(MODEL_PATH, MODEL_NAME))

    # 训练
    model.fit(X, Y, n_epoch=200, validation_set=(X_test, Y_test),
              snapshot_epoch=False, snapshot_step=500,
              show_metric=True, batch_size=16, shuffle=True,
              run_id='resnet_cat_dog')

    # 将训练模型保存
    model.save(pjoin(MODEL_PATH, MODEL_NAME))


if __name__ == '__main__':
    train()

### 预测

In [ ]:
import os

import numpy as np
import tflearn

from image_utils import preprocess_image
from train import resnet

pjoin = os.path.join
MODEL_PATH = pjoin(os.path.dirname(__file__), 'model')
MODEL_NAME = 'resnet_dogs_vs_cats.model'

import tensorflow as tf

tf.app.flags.DEFINE_string('image', None, 'image to classify')
FLAGS = tf.app.flags.FLAGS


def predict(image_file):
    # 读入数据并预处理
    image = preprocess_image(image_file, [256, 256, 3])
    
    # 加载模型
    model = tflearn.DNN(resnet())
    model.load(pjoin(MODEL_PATH, MODEL_NAME))
    
    # 得到预测结果
    y_pred = model.predict([image])
    label = np.argmax(y_pred[0])
    return 'Cat' if label == 0 else 'Dog'


if __name__ == '__main__':
    pred = predict(FLAGS.image)
    print('It\'s a picture of %s!' % pred)